In [165]:
import re, math
from flask import json
import numpy as np
# from pathfinding import construct_lng_lat_matrix, get_min_path
import mercantile
from elevation import getElevationMatrix, rasterToImage, getRasterRGB

MAPBOX_TOKEN = "pk.eyJ1IjoiY3Jpc3BpYW5tIiwiYSI6ImNsMG1oazJhejE0YzAzZHVvd2Z1Zjlhb2YifQ.cv0zlPYY6WnoKM9YLD1lMQ"

In [181]:
def get_tile(lat, lng, zoom_level):
    tile_coords = mercantile.tile(lng=lng, lat=lat, zoom=zoom_level)
    elevation_mat = getElevationMatrix(
        MAPBOX_TOKEN, tile_coords.z, tile_coords.x, tile_coords.y
    )
    padded_mat = np.pad(
        elevation_mat, [(1, 1), (1, 1)], mode="constant", constant_values=np.Inf
    )
    # print(padded_mat, flush=True)
    # Get latitude and longitude at upper-left of tile
    upper_left = mercantile.ul(tile_coords)
    djikstra(
        padded_mat,
        startNode=(1, 1),
        targetNode=(248, 250),
        zoomlevel=zoom_level,
        latitude=lat,
        elevation_multiplier=10,
        show_plot=True,
    )


def construct_lng_lat_matrix(ul, zoomlevel):

    matrix = np.zeros([256, 256], list)

    for i in range(256):
        for j in range(256):
            lng_lat = coord_to_lng_lat(ul, coord=(i, j), zoomlevel=zoomlevel)
            matrix[i, j] = [lng_lat[0], lng_lat[1]]

    return matrix


def coord_to_lng_lat(ul, coord, zoomlevel):
    """Converts x,y matrix coordinate into longitude and latitude coordinates"""
    # Unpack upper-left of tile longitude and latitude
    ul_lat = ul[1]
    ul_lng = ul[0]

    # Calculate distance between each pixel
    latitude_radians = ul_lat * math.pi / 180
    resolution = abs(156543.03 * np.cos(latitude_radians) / (2**zoomlevel))

    # Radius of Earth in metres
    R = 6378137

    # Change in distance (delta pixels * resolution in metres)
    dn = coord[0] * resolution
    de = coord[1] * resolution

    dLat = dn / R
    dLon = de / (R * math.cos(math.pi * ul_lat / 180))

    latO = ul_lat + dLat * 180 / math.pi
    lonO = ul_lng + dLon * 180 / math.pi

    return [lonO, latO]


def lng_lat_to_coord(lng_lat_matrix, lng_lat):

    distances_matrix = np.zeros(
        [lng_lat_matrix.shape[0], lng_lat_matrix.shape[1]], dtype=float
    )
    for i in range(distances_matrix.shape[0]):
        for j in range(distances_matrix.shape[1]):
            distance = abs(sum(np.array(lng_lat_matrix[i, j]) - np.array(lng_lat)))
            distances_matrix[i, j] = distance

    min_idx = np.unravel_index(distances_matrix.argmin(), distances_matrix.shape)

    return min_idx


def djikstra(matrix, startNode, targetNode, resolution, elevation_multiplier=4):

    neighbourDiffs = [[0, 1], [0, -1], [-1, 0], [1, 0]]
    visitedNodes = {
        startNode: 0
    }  # Dictionary of nodes and their shortest discovered cumulative distance
    frontierNodes = dict()
    parentDict = dict()

    currentNode = startNode
    currentDist = 0
    # print("startNode:", startNode)
    # print("targetNode:", targetNode)
    while True:

        neighbourNodes = set(
            tuple(np.array(currentNode) + np.array(diff)) for diff in neighbourDiffs
        )

        for node in neighbourNodes:
            if node not in visitedNodes.keys():
                # Generate weighting for traversing to neighbouring node
                neighbourDist = (
                    currentDist
                    + resolution
                    + elevation_multiplier * abs((matrix[node] - matrix[currentNode]))
                )

                # Update frontier if newly-discovered distance is smaller than existing frontier distance
                try:
                    if neighbourDist < frontierNodes[node]:
                        frontierNodes[node] = neighbourDist
                        # Update parent node for shortest path
                        parentDict[node] = currentNode
                except KeyError:
                    frontierNodes[node] = neighbourDist
                    parentDict[node] = currentNode

        # Search frontier nodes for smallest distance
        smallestFrontierNode = min(frontierNodes, key=frontierNodes.get)

        # Change current node to smallest frontier node
        currentNode = smallestFrontierNode
        currentDist = frontierNodes[currentNode]

        # Remove new current node from frontier
        frontierNodes.pop(currentNode, None)

        # Add new current node to visited nodes
        visitedNodes[currentNode] = currentDist
        if targetNode in visitedNodes.keys():
            # print("DONE")
            break

    # print(len(visitedNodes))

    # Backtracking to get path of nodes
    currentNode = targetNode
    nodePath = [currentNode]
    while currentNode != startNode:
        currentNode = parentDict[currentNode]
        nodePath.append(currentNode)
    # print(nodePath)

    return nodePath

In [167]:
start_click = '{"x":1216,"y":525}<br>{"lng":-2.6192322226170024,"lat":51.44931147369441}'
end_click = '{"x":1216,"y":525}<br>{"lng":-2.614962342949676,"lat":51.44890115167271}'

location1 = json.loads(re.findall("\{.*?\}", start_click)[1])
location2 = json.loads(re.findall("\{.*?\}", end_click)[1])

zoom_level = 14

In [169]:
def convertToJson(minPath):
    geojson = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "properties": {},
                "geometry": {"type": "LineString", "coordinates": minPath},
            }
        ],
    }
    output = open("crispian_Path.geojson", "w")
    json.dump(geojson, output)
    return geojson

In [170]:
# GOOD
start_lng_lat = [location1["lng"], location1["lat"]]
end_lng_lat = [location2["lng"], location2["lat"]]

# route_to_place = get_min_path(start_lng_lat, end_lng_lat, 14)
# minpath = convertToJson(route_to_place)

# minpath

In [38]:
# # BAD
# startPoint = [location1["lat"], location1["lng"]]
# endPoint = [location2["lat"], location2["lng"]]

# route_to_place = get_min_path(startPoint, endPoint, 14)
# minpath = convertToJson(route_to_place)


# minpath["features"][0]["geometry"]["coordinates"][:] = map(
#     lambda l: list(reversed(l)),
#     minpath["features"][0]["geometry"]["coordinates"],
# )
# # minpath

In [82]:
start_lng_lat

[-2.6192322226170024, 51.44931147369441]

In [65]:
end_lng_lat

[-2.614962342949676, 51.44890115167271]

In [171]:
# Get direction of end location relative to start
x_delta = end_lng_lat[0] - start_lng_lat[0]
x_delta

0.004269879667326393

In [172]:
y_delta = end_lng_lat[1] - start_lng_lat[1]
y_delta

-0.00041032202170043774

In [173]:
if (x_delta < 0) and (y_delta > 0):
    tile_lng_lat = end_lng_lat
    startNode = (256, 256)

elif (x_delta < 0) and (y_delta < 0):
    tile_lng_lat = (end_lng_lat[0], start_lng_lat[1])
    startNode = (1, 256)

elif (x_delta > 0) and (y_delta < 0):
    tile_lng_lat = start_lng_lat
    startNode = (1, 1)

elif (x_delta > 0) and (y_delta > 0):
    tile_lng_lat = (start_lng_lat[0], end_lng_lat[1])
    startNode = (256, 1)

print(tile_lng_lat)
print(startNode)

[-2.6192322226170024, 51.44931147369441]
(1, 1)


In [211]:
# Get mercantile tile x,y,z from lng, lat, zoom
tile_coords = mercantile.tile(lng=tile_lng_lat[0], lat=tile_lng_lat[1], zoom=14)
print('tile_coords:', tile_coords)
upper_left = mercantile.ul(tile_coords)
print('\nupper_left: ', upper_left)

tile_coords: Tile(x=8072, y=5452, z=14)

upper_left:  LngLat(lng=-2.63671875, lat=51.45400691005982)


In [231]:
upper_left.lat

51.45400691005982

In [222]:
lng_lat_matrix = construct_lng_lat_matrix(upper_left, zoomlevel=14)
lng_lat_matrix

array([[list([-2.63671875, 51.45400691005982]),
        list([-2.6366329193136773, 51.45400691005982]),
        list([-2.636547088627354, 51.45400691005982]), ...,
        list([-2.615003586360315, 51.45400691005982]),
        list([-2.6149177556739924, 51.45400691005982]),
        list([-2.6148319249876697, 51.45400691005982])],
       [list([-2.63671875, 51.45406039482198]),
        list([-2.6366329193136773, 51.45406039482198]),
        list([-2.636547088627354, 51.45406039482198]), ...,
        list([-2.615003586360315, 51.45406039482198]),
        list([-2.6149177556739924, 51.45406039482198]),
        list([-2.6148319249876697, 51.45406039482198])],
       [list([-2.63671875, 51.45411387958414]),
        list([-2.6366329193136773, 51.45411387958414]),
        list([-2.636547088627354, 51.45411387958414]), ...,
        list([-2.615003586360315, 51.45411387958414]),
        list([-2.6149177556739924, 51.45411387958414]),
        list([-2.6148319249876697, 51.45411387958414])],
    

In [223]:
targetNode = lng_lat_to_coord(lng_lat_matrix, lng_lat=list(end_lng_lat))
targetNode
#print("startNode", startNode)
#print("targetNode", targetNode)
# Get elevation matrix

(138, 108)

In [224]:
elevation_mat = getElevationMatrix(MAPBOX_TOKEN, tile_coords.z, tile_coords.x, tile_coords.y)
elevation_mat

array([[109.1, 109.2, 109.3, ...,  76.4,  76.5,  76.6],
       [109.4, 109.5, 109.6, ...,  76.6,  76.7,  76.8],
       [109.8, 109.9, 110. , ...,  76.7,  76.8,  77. ],
       ...,
       [ 17.7,  17.8,  17.9, ...,   9.3,   9.4,   9.5],
       [ 17.3,  17.5,  17.6, ...,   9.2,   9.2,   9.3],
       [ 17.1,  17.3,  17.4, ...,   9. ,   9.1,   9.1]])

In [225]:
# Pad matrix with infinities to represent boundaries
padded_mat = np.pad(elevation_mat, [(1, 1), (1, 1)], mode='constant', constant_values=np.Inf)
padded_mat

array([[  inf,   inf,   inf, ...,   inf,   inf,   inf],
       [  inf, 109.1, 109.2, ...,  76.5,  76.6,   inf],
       [  inf, 109.4, 109.5, ...,  76.7,  76.8,   inf],
       ...,
       [  inf,  17.3,  17.5, ...,   9.2,   9.3,   inf],
       [  inf,  17.1,  17.3, ...,   9.1,   9.1,   inf],
       [  inf,   inf,   inf, ...,   inf,   inf,   inf]])

In [226]:
# resolution = 156543.03 meters/pixel * cos(latitude) / (2 ^ zoomlevel)
latitude_radians = tile_lng_lat[1] * math.pi / 180
resolution = abs(156543.03 * np.cos(latitude_radians) / (2 ** 14))

print("latitude_radians", latitude_radians)
print("resolution", resolution)

latitude_radians 0.8979598831000634
resolution 5.954508867186662


In [227]:
# Generate the shortest path as a sequence of lng, lat tuples
node_path = djikstra(padded_mat, startNode=startNode, targetNode=(targetNode[0]+1, targetNode[1]+1),
                        resolution=resolution, elevation_multiplier=5)
# node_path

In [228]:
# Get lng and lat of upper-left of tile
upper_left = mercantile.ul(tile_coords)
upper_left

LngLat(lng=-2.63671875, lat=51.45400691005982)

In [229]:
# Gets path as series of longitude and latitude coordinates
lnglatPath = [coord_to_lng_lat(upper_left, coord, 14) for coord in node_path]
lnglatPath
# if show_img:
#     plt.imshow(elevation_mat, interpolation='nearest')
#     xs = [x[0] for x in node_path]
#     ys = [x[1] for x in node_path]
#     plt.plot(xs, ys, 'r-')
#     plt.show()

minpath = convertToJson(lnglatPath)

In [207]:
# def coord_to_lng_lat(ul, coord, zoomlevel):

ul = tile_lng_lat
coord = [0, 1]

"""Converts x,y matrix coordinate into longitude and latitude coordinates"""
# Unpack upper-left of tile longitude and latitude
ul_lat = ul[1]
print("ul_lat", ul_lat)
ul_lng = ul[0]
print("ul_lng", ul_lng)
# Calculate distance between each pixel
latitude_radians = ul_lat * math.pi / 180
print("latitude_radians", latitude_radians)
resolution = abs(156543.03 * np.cos(latitude_radians) / (2**14))
print("resolution", resolution)

# Radius of Earth in metres
R = 6378137

# Change in distance (delta pixels * resolution in metres)


ul_lat 51.44931147369441
ul_lng -2.6192322226170024
latitude_radians 0.8979598831000634
resolution 5.954508867186662


In [208]:
dn = coord[0] * resolution
print("dn: ", dn)
de = coord[1] * resolution
print("de: ", de)

dLat = dn / R
print("dLat: ", dLat)
dLon = de / (R * math.cos(math.pi * ul_lat / 180))
print("dLon: ", dLon)


dn:  0.0
de:  5.954508867186662
dLat:  0.0
dLon:  1.4980280755804458e-06


In [209]:
latO = ul_lat + dLat * 180 / math.pi
latO

51.44931147369441

In [210]:
lonO = ul_lng + dLon * 180 / math.pi
lonO

-2.6191463919306797